# Simulation of Drug Transport across a Virtual Skin Membrane

### Initialize UG4 (for 2D and standard algebra)

In [30]:
InitUG(2, AlgebraType("CPU", 1));
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")

### Create Domain

In [31]:
requiredSubsets = {"LIP", "COR", "BOTTOM_SC", "TOP_SC"}
gridName = "skin2d-aniso.ugx"
numRefs = 2

In [32]:
dom = util.CreateDomain(gridName, numRefs, requiredSubsets)

Loading Domain skin2d-aniso.ugx ... done.
Performing integrity check on domain ... done.
Refining(2): 1 2 done.


### Create Approximation space

In [33]:
approxSpaceDesc = { fct = "c", type = "Lagrange", order = 1 }

In [34]:
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct(approxSpaceDesc.fct, approxSpaceDesc.type, approxSpaceDesc.order)
approxSpace:init_levels()
approxSpace:init_top_surface()
print("Approximation space:")
approxSpace:print_statistic()

Approximation space:
| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |       0: LIP |       1: COR | 2: BOTTOM_SC |    3: TOP_SC |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |          680 |           32 |          608 |           20 |           20 |
| (lev,    1)    |         2613 |          783 |         1752 |           39 |           39 |
| (lev,    2)    |        10241 |         4367 |         5720 |           77 |           77 |
| (lev,    0, g) |          680 |           32 |          608 |           20 |           20 |
| (lev,    1, g) |         2613 |      

## Create a convection-diffusion-equation
Model parameter

In [35]:
K={
    ["LIP"] = 1.0, ["COR"] = 1.0,
}

D={
     ["LIP"] = 1, ["COR"] = 0.01, 
}


In [36]:
elemDisc ={}

elemDisc["COR"] = ConvectionDiffusion("c", "COR", "fv1")
elemDisc["COR"]:set_diffusion(K["COR"]*D["COR"])
elemDisc["COR"]:set_mass_scale(K["COR"])

elemDisc["LIP"] = ConvectionDiffusion("c", "LIP", "fv1")
elemDisc["LIP"]:set_diffusion(K["LIP"]*D["LIP"])
elemDisc["LIP"]:set_mass_scale(K["LIP"])

In [37]:
dirichletBnd = DirichletBoundary()
dirichletBnd:add(1.0, "c", "TOP_SC")
dirichletBnd:add(0.0, "c", "BOTTOM_SC")

In [38]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc["LIP"])
domainDisc:add(elemDisc["COR"])
domainDisc:add(dirichletBnd)

## Create  solver

In [39]:
lsolver=SuperLU()

## Solve transient problem

In [40]:
u = GridFunction(approxSpace)
u:set(0.0)

In [50]:
startTime = 0.0
endTime = 10.0
dt=0.25

In [42]:
util.SolveLinearTimeProblem(u, domainDisc, lsolver, VTKOutput(), "vtk/skin",
                            "ImplEuler", 1, startTime, endTime, dt); 

SolveLinearTimeProblem, Linear Solver setup:
SuperLU
>> Writing start values
++++++ TIMESTEP 1 BEGIN (current time: 0) ++++++
++++++ Time step size: 0.25
++++++ Assembling Matrix/Rhs for step size 0.25
++++++ TIMESTEP 1 END   (current time: 0.25) ++++++
++++++ TIMESTEP 2 BEGIN (current time: 0.25) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 2 END   (current time: 0.5) ++++++
++++++ TIMESTEP 3 BEGIN (current time: 0.5) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 3 END   (current time: 0.75) ++++++
++++++ TIMESTEP 4 BEGIN (current time: 0.75) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 4 END   (current time: 1) ++++++
++++++ TIMESTEP 5 BEGIN (current time: 1) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 5 END   (current time: 1.25) ++++++
++++++ TIMESTEP 6 BEGIN (current time: 1.25) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 6 END   (current time: 1.5) ++++++
++++++ TIMESTEP 7 BEGIN (current time: 1.5) ++++++
++++++ Time step size: 0.25
++++++ TIMESTEP 

In [48]:
timeDisc=ThetaTimeStep(domainDisc, 1.0)

timeInt = LinearTimeIntegrator(timeDisc)
timeInt:set_linear_solver(lsolver)
timeInt:set_time_step(0.25)


In [49]:
timeInt:apply(u, endTime, u, startTime)

+++ Integrating: [0, 10]
+++ Timestep +++1
+++ Reassemble (t=0, dt=0.25)
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++2
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++3
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++4
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++5
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++6
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++7
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++8
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++9
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++10
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d138918
+++ Timestep +++11
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f905d1389

In [46]:
timeInt = ConstStepLinearTimeIntegrator(timeDisc)
timeInt:set_linear_solver(lsolver)
timeInt:set_time_step(0.25)


In [47]:
timeInt:apply(u, endTime, u, startTime)

+++ Integrating: [	0	, 	10	] with dt=	0.25(40 iters)
+++ Const timestep +++1t=0->0.25
TimeIntegratorSubject::notify_group[0]: 0 observers. 0x7f9064d97678
+++ Assemble (t=0, dt=0.25)
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f9064d97678
+++ Const timestep +++2t=0.25->0.25
TimeIntegratorSubject::notify_group[0]: 0 observers. 0x7f9064d97678
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f9064d97678
+++ Const timestep +++3t=0.5->0.25
TimeIntegratorSubject::notify_group[0]: 0 observers. 0x7f9064d97678
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f9064d97678
+++ Const timestep +++4t=0.75->0.25
TimeIntegratorSubject::notify_group[0]: 0 observers. 0x7f9064d97678
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f9064d97678
+++ Const timestep +++5t=1->0.25
TimeIntegratorSubject::notify_group[0]: 0 observers. 0x7f9064d97678
TimeIntegratorSubject::notify_group[2]: 0 observers. 0x7f9064d97678
+++ Const timestep +++6t=1.25->0.25
TimeIntegratorSubject::notify_